<a href="https://colab.research.google.com/github/tumerkanaldanmaz1/e-commerce/blob/main/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Gerekli kütüphane
from google.colab import files
import pandas as pd

# Dosya yükleme kutusunu aç
uploaded = files.upload()

Saving data.csv to data.csv


In [2]:
# Yüklenen dosyanın adını otomatik al
file_name = next(iter(uploaded))

# CSV'yi oku (Türkçe karakterler için özel encoding)
df = pd.read_csv(file_name, encoding='ISO-8859-9')

# İlk 5 satırı göster
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [3]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df.dtypes  # kontrol etmek için

,0
InvoiceNo,object
StockCode,object
Description,object
Quantity,int64
InvoiceDate,datetime64[ns]
UnitPrice,float64
CustomerID,float64
Country,object


In [4]:
df = df.dropna(subset=['CustomerID'])
df['CustomerID'] = df['CustomerID'].astype(int)
df.dtypes

<ipython-input-4-1cbb8ea4b903>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CustomerID'] = df['CustomerID'].astype(int)


,0
InvoiceNo,object
StockCode,object
Description,object
Quantity,int64
InvoiceDate,datetime64[ns]
UnitPrice,float64
CustomerID,int64
Country,object


In [6]:
df['IsCancelled'] = df['Quantity'] < 0
df_sales = df[~df['IsCancelled']] # Sadece satışlar (iptal olmayanlar)
df_sales.head() # Kontrol için ilk 5 satır

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,IsCancelled
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,False
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,False
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,False
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,False
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,False


In [7]:
!pip install -q pandasql
from pandasql import sqldf

# SQL sorgularını çalıştırmak için kısa yol
pysqldf = lambda q: sqldf(q, globals())

  Preparing metadata (setup.py) ... done


In [8]:
query1 = """
SELECT Description,SUM(Quantity) AS TotalQuantity
FROM df_sales
GROUP BY Description
ORDER BY TotalQuantity DESC
LIMIT 10
"""
topproducts = pysqldf(query1)
topproducts

,Description,TotalQuantity
0,"PAPER CRAFT , LITTLE BIRDIE",80995
1,MEDIUM CERAMIC TOP STORAGE JAR,77916
2,WORLD WAR 2 GLIDERS ASSTD DESIGNS,54415
3,JUMBO BAG RED RETROSPOT,46181
4,WHITE HANGING HEART T-LIGHT HOLDER,36725
5,ASSORTED COLOUR BIRD ORNAMENT,35362
6,PACK OF 72 RETROSPOT CAKE CASES,33693
7,POPCORN HOLDER,30931
8,RABBIT NIGHT LIGHT,27202
9,MINI PAINT SET VINTAGE,26076


In [9]:
##hangi ülkeye ne kadar satış yapılmış onu gösterir

query2 = """

SELECT Country,SUM(Quantity) AS TotalQuantity
FROM df_sales
GROUP BY Country
ORDER BY TotalQuantity DESC
"""
country_sales = pysqldf(query2)
country_sales

,Country,TotalQuantity
0,United Kingdom,4269472
1,Netherlands,200937
2,EIRE,140525
3,Germany,119263
4,France,111472
5,Australia,84209
6,Sweden,36083
7,Switzerland,30083
8,Spain,27951
9,Japan,26016


In [12]:
##en çok harcama yapan müşteriler

query3 = """

SELECT CustomerID,SUM(Quantity * UnitPrice ) AS TotalSpent
FROM df_sales
GROUP BY CustomerID
ORDER BY TotalSpent DESC
LIMIT 10
"""
topcustomers = pysqldf(query3)
topcustomers

,CustomerID,TotalSpent
0,14646,280206.02
1,18102,259657.30
2,17450,194550.79
3,16446,168472.50
4,14911,143825.06
5,12415,124914.53
6,14156,117379.63
7,17511,91062.38
8,16029,81024.84
9,12346,77183.60


In [13]:
# Tarih sütununu günlük olarak ayıralım
df_sales['Date'] = df_sales['InvoiceDate'].dt.date

# Toplam harcamayı hesapla
df_sales['TotalPrice'] = df_sales['Quantity'] * df_sales['UnitPrice']

# Sadece gerekli sütunları alalım
df_powerbi = df_sales[['InvoiceNo', 'Date', 'CustomerID', 'Country', 'Description', 'Quantity', 'UnitPrice', 'TotalPrice']]

# CSV'ye kaydet
df_powerbi.to_csv('powerbi_data.csv', index=False)

# İndirilebilir hali
from google.colab import files
files.download('powerbi_data.csv')


<ipython-input-13-78f7be344a3f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sales['Date'] = df_sales['InvoiceDate'].dt.date
<ipython-input-13-78f7be344a3f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sales['TotalPrice'] = df_sales['Quantity'] * df_sales['UnitPrice']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>